# Matching 20201105

Compares a list of masses and labels generated with CompoundCalculator to an input peak list. It is convenient to open the Calculator and Match notebooks in side-by-side windows (Jupyter Lab allows this) so it is easy to update the target ion list and repeat the matching.

## Overview

The program matches the target ion and peak lists using a tolerance that can be specified in amu (fixed( or ppm (increases with mass). If both are non-zero the ppm is calulated and the larger window used which allows the window to increase with mass but never be less that a certain value. The program also allows for the possibility that complex peak and target ion lists can result in multiple matches for each peak. The function that prints the matches has a simplify mode which shows only one match for each peak but appends a string showing the number of matches; the match shown is the one with the smallest absolute error. Matched peaks can be grouped according to the 'root' field, which is part of the target ion list, and there is a cell that shows peaks with redundant matches to emphasize their existence and allow adjustment of the matching or ion generation parameters if necessary.

Unmatched peaks greater than a given intensity threshold (percent base peak intensity) are also displayed. The idea is that this is part of interactive spectrum interpretation, i.e. once the origin of unmatched peaks is understood the ion generation parameters can be adjusted and applied to other peaks.

In order to ensure that isotope peaks stay with the monoisotopic peaks, the program only searches for 13C peaks for matched peaks. Identified isotope peaks are Peaks identified as isotopes can also match entries in the target ion list so other possibilities are shown. In the output lists each peak has an index as well as the index of a related monoisotopic peak; these are the same for actual monoisotopic peaks.

The peak list must be tab-delimited and have mass values but can also contain columns for Retention Time (RT) and Intensity; the function that reads the peak list tries to determine which columns are present.

Results can be saved in several ways including a simple mass/intensity list and more detailed lists. Text lists can be imported into PeakView as spectra and overlaid on the original data to visualize the matches and highlight unmatched peaks.

## Imports and function definitions

In [1]:
import os
import datetime 
from collections import namedtuple
from itertools import groupby

In [2]:
# Define some named tuples to hold the data and results
Peak = namedtuple('Peak', 'Mass Inten RT')
Target = namedtuple('Target', 'Mass Root Label')

# a match contains the index of the peak, the calculated ion it matches, the index of the monoisotopic peak (if applicable) and the mass error
# the latter fields allow the matched lists to be redily sorted and filtered
Match = namedtuple('Match', 'Pk_index TargetIon MonoPeak Error')   

In [3]:
def values_from_line(line):
    """
    Split a line into parts and try to convert them to numbers...return the list of numbers or fields
    """
    
    parts = line.split()
    
    try:
        vals = [float(field) for field in parts]  # convert all to numbers
        success = True
    except:
        vals = parts      # return the fields if the conversion fails
        success = False
    
    return success, vals
    
    
def read_peak_list(peak_file_path):
    """
    Reads a tab-delimited text file generating a list of Peak tuples (Mass, Inten, RT). Mass must be present but the other fields are optional
    and will be stored internally as zero if absent.
    If the file has only one column it is assumed to contain masses otherwise the code assumes that the first column is Mass,
    the second is Inten and the RT is absent.
    If the file contains a header line it is used to define the order of the columns by looking for matches with common labels
    e.g. mz, m/z, Mass, etc. for masses. The RT column can only be used via a header line containi 'RT' or 'rt'
    """
    mass_col = 0
    inten_col = 1
    rt_col = -1
    has_rt = False
    has_inten = True
    start_line= 0
    
    peaks = []    #list of (mass, inten, RT) tuples

    # read all the lines so we can process them one-by-one
    with open(peak_file_path, 'r') as f:  
    
        lines = f.readlines()
        
        f.close()
    
    success, vals = values_from_line(lines[0])  # try to convert the first line
    
    if not success:   # couldn't get values; probably a header....vals is a list of the parts

        # see if we can figure out what the columns are...lsist can be extende if needed
        for col_index, col in enumerate(vals):
            if col in ['mz', 'm/z', 'mass', 'Mass', 'Mass/Charge']: mass_col = col_index
            if col in ['Int', 'Inten', 'inten', 'Height']: inten_col = col_index
            if col in ['RT', 'rt']: rt_col = col_index

        has_rt = rt_col > -1
        has_inten = inten_col > -1
            
        start_line = 1     # can skip this line

        print('m:', mass_col, 'Int:', inten_col, 'RT:', rt_col)

    # Process line by line. Lines that cannot be converted to numbers are reported 
    # Note: the first line will be reprocessed if it is numeric
    for line in lines[start_line:]:        

        # get a list of numbers from the fields in the line - success will be false if this fails and vals will be the actual text parts
        success, vals = values_from_line(line)  
               
        if success:
            mass = vals[mass_col]
            rt = vals[rt_col] if has_rt else 0
            inten = vals[inten_col] if has_inten else 0
            
            p = Peak(mass, inten, rt)
              
            peaks.append(p) 
            
        else:
            print('Problem in line:', line, vals)   # vals will be the list of fields if there's a problem
    
    peaks = sorted(peaks, key = lambda x: x.Mass)     # ensure the list is sorted by mass

    masses, intens, rts = zip(*peaks)

    # Note: intensity params will be 0 if there is no intensity column
    return peaks, sum(intens), max(intens), has_rt    # peaks, tic, base_peak_inten, sum rts

In [4]:
def read_ion_list(ion_file_path):
    """
    Read the taget ion file which is assumed to contain fields for mass, root and label that can be convertes to a Target
    """
    
    ions = []            #list of (mass, root, label) tuples
    cond_str = ""        # conditions line if ptrsdrnt
    
    with open(ion_file_path, 'r') as f:  
    
        for line in f:
            
            if line[0] == '#':             # if the first character is '#' this is a condition line
                cond_str = line[1:]
                continue
            
            parts = line.split()
            
            try:
                ion = Target(float(parts[0]), parts[1], parts[2])
                ions.append(ion) 
            except:
                print('Problem in', line)

        f.close()
    
    ions = sorted(ions, key = lambda x: x.Mass)     # ensure the list is sorted by mass

    return ions, cond_str    # target ions

In [5]:
def get_mz_window(mz, amu, ppm):
    """
    Determine the mass window to use for matching either as ppm or amu.
    If both are supplied the larger will be used; this allows the window to increase with mass (ppm) but
    never be smaller than amu.
    If either is zero, the other is automatically used    
    """
    ppm_window = mz * ppm /1e6
    
    if ppm_window > amu:
        return ppm_window
    else:
        return amu
    
def get_mass_limits(mz, amu, ppm):
    """
    Uses get_mz_window to determine the window size and returns the upper an lower mass limits
    """
    window = get_mz_window(mz, amu, ppm)
    return mz - window, mz + window

# demonstartes how the window varies with mass
# for mz in [100,200,300,400,500,600,700,800,900,1000]:
#     print(mz, get_mz_window(mz, 0.005, 15), get_mass_limits(mz, 0.005, 15))


In [6]:
def get_match_stats(matches, peaks, tic):
    """
    Given lists of peaks and matches, returns the indices of the matched peaks and the percentage
    of the TIC that they explain
    """

    matched_indices = set([m.Pk_index for m in matches])   # get the peak indices; we uae a set so each peak occurs only once

    matched_inten = sum([peaks[i].Inten for i in matched_indices])  # sum the intensities

    percent_matched = matched_inten * 100/tic
    
    return matched_indices, percent_matched

In [7]:
def get_redundant_matches(matches):
    """
    Find which peaks have multiple matches.
    We group the matches by peak index and then find which groups have more than one entry
    We return the count and the redundant groups individually sorted by absolute error
    """
    redundant = []
    redundant_peak_count = 0   # Number of peaks with more than one match
    
    m_sorted = sorted(matches, key=lambda x: x.Pk_index)
    m_grps = groupby(m_sorted, lambda x: x.Pk_index)
    
    for k, grp in m_grps:
        
        # to get the group as a list sorted by the length of the label use the following
#       grp_as_list = sorted(list(grp), key= lambda x: len(x[1].Label))
        
        # get the group as a list sorted by absolute error
        grp_as_list = sorted(list(grp), key= lambda x: abs(x.Error))

        if len(grp_as_list) > 1:
            redundant += grp_as_list
            redundant_peak_count += 1
        
    return redundant_peak_count, redundant
        

In [8]:
def get_unmatched_indices(matched_indices, peaks, threshold):
    """
    Find the indices of unmatched peaks that exceed a threshold
    """    
    # get a boolean list indicating which peak is matched
    peak_matches = [True if i in matched_indices else False for i in range(len(peaks))]
    
    # now find which ones ae false
    unmatched = [i for i in range(len(peaks)) if not peak_matches[i]]
    
    return [i for i in unmatched if peaks[i].Inten >= threshold]

In [9]:
def save_matches_to_file(out_path, matches_to_save, peaks, conditions, match_conditions, with_details=False):
    """
    Save the list of matches to a file. We use one of the print functions to do this
    """
    with open(out_path, 'w') as f: 
        
        # combine the conditions and match conditions in a single string
        match_conds_str = ';'.join(match_conditions)
        
        if conditions:
            conds =  ';'.join([conditions.rstrip(), match_conds_str])
        else:
            conds = match_conds_str

        print(conds, file=f)
            
        if with_details:
            print(match_with_tabs_header(), file=f)
            for m in matches_to_save:         
                print(match_as_str_with_tabs(m, peaks), file=f)
        else:
            print(match_short_with_tabs_header(), file=f)
            for m in matches_to_save: 
                print(match_as_short_str_with_tabs(m, peaks), file=f)
                
        f.close()

    return(len(to_save))


In [10]:
# Functions that convert a match to various output strings
def match_as_str(m, peaks):
    """
    Prettified string for printing in a Notebook cell
    """    
    p_index, ion, mono_pk, error = m         # Unpack the peak index and the matching composition
    p = peaks[p_index]       # peak mass and intensity
    
    error = error * 1000   # error in mmu   
    
    return f'{p_index:5}:{p.Mass:10.4f} ({error:5.1f}) {p.RT:6.2f} {p.Inten:12.1f} {ion.Mass:10.4f} {mono_pk:6}  {ion.Root:14}{ion.Label}'

def match_as_str_with_tabs(m, peaks):
    """
   Tab delimited detailed string.
    """    
    p_index, ion, mono_pk, error   = m       # Unpack the peak index and the matching composition
    p = peaks[p_index]       # peak mass and intensity
    
    error = error * 1000   # error in mmu   
        
    return f'{p.Mass:.4f}\t{p.Inten:.1f}\t{error:.1f}\t{p.RT:.1f}\t{p_index}\t{mono_pk}\t{ion.Mass:.4f}\t{ion.Root:12}\t{ion.Label}' 

def match_with_tabs_header():
    
    return "Pk_mass\tPk_inten\tDelta_mmu\tPk_RT\tPk_index\tMono__pk\tMatch_mass\tMatch_root\tMatch_label"

def match_as_short_str(m, peaks):
    """
    Simplified pretty string
    """
    p_index, ion, mono_pk, error = m         # Unpack the peak index and the matching composition
    p = peaks[p_index]       # peak mass and intensity
        
    return f'{p.Mass:10.4f} {p.Inten:12.1f} {p.RT:6.2f} {ion.Root} {ion.Label}'

def match_as_short_str_with_tabs(m, peaks):
    """
    Tab delimited mass and intensity for use elsewhere
    """
    p_index, ion, mono_pk, error = m         # Unpack the peak index and the matching composition
    p = peaks[p_index]       # peak mass and intensity
        
    return f'{p.Mass:.4f}\t{p.Inten:.1f}\t{p.RT:.2f}\t{ion.Root}\t{ion.Label}'

def match_short_with_tabs_header():
    
    return "Pk_mass\tPk_inten\tRT\tRoot\tMatch_label"

In [11]:
def print_match_list(matches, peaks, print_fn, simplify=False):    
    """
    Use the provided print function (e.g. match_as_str) to print the matches provided
    If simplify == True, we only print one entry and append a string to indicate there are more
    If group_first is True, we group the results by 'Root' before printing
    """
    m_sorted = sorted(matches, key=lambda x: x.Pk_index)
    m_grps = groupby(m_sorted, lambda x: x.Pk_index)     # group by peak index...groups with more than one entry have redundancy
    
    for k, grp in m_grps:
        
        # get the group as a list sorted by absolute error
        grp_as_list = sorted(list(grp), key= lambda x: abs(x.Error))

        
        simplifying = simplify and (len(grp_as_list) > 1)   # do we need to simplify the output?
        for g in grp_as_list:
   
            desc = print_fn(g, peaks)   
                
            if simplifying:
                desc += f' [1/{len(grp_as_list)}]'
            
            print(desc)
            
            if simplifying: break  #only print one line
        
    print()

In [12]:
# a utility function to help sort peak lists
def get_peak_mass_from_match(m, peaks):    
    return peaks[m.Pk_index].Mass

def get_count_and_tic(grp, peaks):
    """
    Given a group of matches determine the number of unique peaks and the sum of the intensity they explain
    """
    
    last_index = 0
    count = 0
    tic = 0
    
    for m in grp:
        
        if m.Pk_index == last_index: continue
        
        last_index = m.Pk_index
        
        count += 1
        tic += peaks[last_index].Inten
        
    return count, tic
    
def print_root_groups(matches, peaks, print_fn, suppress_isotopes=False, suppress_print=False):    
    """
    Use the provided print function (e.g. match_as_str) to print the matches provided
    We group the matches by the root first and sort by mass; if suppress_isotopes is True we skip
    isotope matches (i.e. those where the Pk_index and MonoPeak are different)
    Also returns a dictionary of group:(count, tic) that summarizes each group
    """
    
    r_sorted = sorted(matches, key=lambda x: x.TargetIon.Root)
    r_grps = groupby(r_sorted, lambda x: x.TargetIon.Root)     # group by target root
    
    result = {}
    
    for k, grp in r_grps:
        
        # get the group as a list sorted by length of mass
        grp_as_list = sorted(list(grp), key= lambda x: get_peak_mass_from_match(x, peaks))
    
        count, tic = get_count_and_tic(grp_as_list, peaks)
        
        if not suppress_print:
            
            # we can suppress isotopes by only selecting matches where the peak index and mono peak index are the same
            if suppress_isotopes:  
                grp_as_list = [m for m in grp_as_list if m.Pk_index == m.MonoPeak]
            
            for g in grp_as_list:          
                print(print_fn(g, peaks))
         
            print()    # blank line after each group       
        
        result[k] = (count, tic)
      
    return result
            

In [13]:
# Convert a list of limits, i.e. (comp, max count) tuples, to a string
# skip any with max_count = 0 and join the rest with commas
def limits_as_string(limits):
    non_zero_limits = [l for l in limits if l[1] > 0]
    if len(non_zero_limits) == 0:
        return ""
    else:
        desc = ",".join([f'{l}' for l in non_zero_limits])
        return desc

In [14]:
# split the conditions line and print each part on a separate line..
def print_conditions(ion_file, conds_line):
    
    print(ion_file)
    
    parts = conds_line.split(';')
    
    for p in parts:
        print(p.strip())

## Step 1 - Setup

Define paths to the peak list and target ion list. Here we use a directory that is shared with the calculator to facilitate interactive use (we can run the calculator notebook, switch to one this and re-run it to see the changes). The code tests to see if we are using CoLab and, if so, mounts a GDrive and points a data_path at it.

The lines marked 'UPDATE!' must be changed to relect the local environment. Windows users need to specify the disk, i.e.

       data_path = 'C:' + os.sep + os.path.join('Users','ronbonner','Data', 'SharedData'

In [15]:
# Define the path for data files; this allows the Calculator and Match notebooks to easily share data
# We check for Colab and either use a GDrive directory or a local one

try:
    from google.colab import drive
    
    print('Using Colab')
    
    drive.mount('/content/drive')   #  This is user-independent

    data_path = os.sep + os.path.join('content', 'drive', 'MyDrive', 'SharedData')     # UPDATE!
except:
    print('Not using Colab')
    data_path = os.sep + os.path.join('Users','ronbonner','Data', 'SharedData')      # UPDATE!

print(data_path)


Not using Colab
/Users/ronbonner/Data/SharedData


In [16]:
save_matches = True                # do we want to save the matched peaks (as mass, inten, match name)?
local_files = False
include_large_unmatched = False      # do we want to include the larger unmatched peaks (by default > 1% base peak inten)
include_date_in_file_name = False

# the match window can be in amu or ppm (relative to mass); if both are speciefied the larger (at any mass) is used
amu_window = 0.005     # amu half window for peak matching
ppm_window = 20        # ppm half window for peak matching

# We only look for the 13C isotopes of matched peaks
c13_half_window = 0.003     # for matching 13C isotopes
max_C13_count = 4           # maximum number of 13C's to look for
c13_rt_window = 0.2         # main matched peaks and isotopes must have RTs that differ by less than this
require_lower_c13_inten = True  # if True, potential isotope peaks must have a lower intensity than the matched peak

In [17]:
# Read the peak file

peak_file = '200325 SJ C18 Pos A, C001-U.txt'
peak_file_path = peak_file if local_files else os.path.join(data_path, peak_file)

peaks, tic, base_peak_inten, has_RT = read_peak_list(peak_file_path)

rt_str = "has rt" if has_RT else ""

print(peak_file_path)
print(f'{len(peaks)}, peaks read. TIC {tic:.1f}, base peak inten {base_peak_inten} {rt_str}')


m: 0 Int: 2 RT: 1
/Users/ronbonner/Data/SharedData/200325 SJ C18 Pos A, C001-U.txt
540, peaks read. TIC 372780.1, base peak inten 49781.4 has rt


In [18]:
# Read the target ion file

ion_file =  'Ibu ions pos 1.txt'   #
compounds_as_string = 'Ibu'

ion_path = ion_file if local_files else os.path.join(data_path, ion_file)

ions, conditions = read_ion_list(ion_path)

print(ion_path)

print(f'{len(ions)} target ions read')

if conditions:
    print_conditions(ion_file, conditions)


/Users/ronbonner/Data/SharedData/Ibu ions pos 1.txt
360 target ions read
Ibu ions pos 1.txt
Time:201208_092529
Compounds:Ibu
Multimer_limit:3
Polarity:positive
Phase_1:('OH', 2),('COOH', 1)
Phase_2:('gluc', 1)
Adducts:('Na-H', 2),('K-H', 2)
Max adduct count:6
Losses:('H2O', 1)


## Step 1 - Match ions

We first match the ions generated by the calculator. In a subsequent step we look specifically for the 13C forms of matched peaks.

The code allows for the possibility that any peak may match multiple targets, and any target may be matched by multple peaks. This is can occur if retention times are present (e.g. if there are isomers at different RT), if there are very close masses (e.g. from very high resolution data) or if there are many close target ions generated from complex sets of adducts, losses, heterodimers, etc.

In [19]:
# Get the current time for use in the peak name and conditions
current_time = datetime.datetime.now().replace(microsecond=0)
curr_time_str = current_time.strftime('%y%m%d_%H%M%S')

peak_index, ion_index, peaks_matched = 0, 0, 0

matches = []   # this is going to end up as a list of Match tuples : (peak index, matched target)

# Loop all the values and peaks looking for matches within the specified window
while (ion_index < len(ions)) and (peak_index < len(peaks)):

    this_peak, this_ion = peaks[peak_index], ions[ion_index]
    low_peak, high_peak = get_mass_limits(this_peak.Mass, amu_window, ppm_window)
    
    # Increment the ion index if its Mass is too low and the peak if it's too high
    if this_ion.Mass < low_peak:
        ion_index += 1
        continue

    if this_ion.Mass > high_peak:
        peak_index += 1
        continue

    # save peak index and ion composition
    # since there may be more than one peak that matches this ion value, we look ahead at the peaks
    # using a separate index so the current peak can be used with the next ion value
    # we also track the ions matched since some ions may have more than one matching peak
    
    matches.append(Match(peak_index, this_ion, peak_index, this_peak.Mass - this_ion.Mass))    # reference to peak, this composition and the monopeak (this one)
    peaks_matched += 1   
    
    look_ahead = peak_index + 1
 
    # look ahead at the peaks while they're still within the search window and add any matches to the list
    while (look_ahead < len(peaks)):
                
        look_ahead_peak = peaks[look_ahead]
        
        if(look_ahead_peak.Mass - this_ion.Mass) > get_mz_window(this_peak.Mass, amu_window, ppm_window):
            break
            
        matches.append(Match(look_ahead, this_ion, look_ahead, look_ahead_peak.Mass - this_ion.Mass))
        look_ahead +=1
        peaks_matched += 1 


    ion_index += 1 # increment ion index but not peak_index - there may be more than one ion within the window..

matched_indices, percent_tic_matched = get_match_stats(matches, peaks, tic)

matched_indices = sorted(matched_indices)
initial_matches = f'{len(matched_indices)} peaks matched ({percent_tic_matched:.1f}% tic), {len(matches)} total matches from {len(peaks)} peaks'
print(initial_matches)

80 peaks matched (21.5% tic), 82 total matches from 540 peaks


In [20]:
# Look for C13 isotopes of matched peaks
# In addition to looking for mass deltas of 1.003, we test that the retention time differences are within a specified tolerance
# we can always apply this test since missing RTs are stored as zero so the delta is guaranteed to be less than any positive, non-zero tolerance
# We can optionally require that the intensity of isotope peaks be less than the initial matched peak
# We look at the peak indices since we only need to test a peak once, even if it has multiple matches

c13_matches = []

last_matched_mass = 0
last_peak_index = -1

# make sure the peaks are in peak (= mass) order
matches = sorted(matches, key=lambda x: x.Pk_index)

for m in matches:    
        
    if m.Pk_index == last_peak_index:    #only need to look at each peak once
        continue

    peak_index = m.Pk_index    
    last_peak_index = peak_index
        
    m_mass, m_inten, m_rt = peaks[peak_index]     # get the matched peak...
    
    next_peak_index = peak_index      #...and start looking for isotopes at the next higher peak
        
    keep_going = True
    
    for c13_count in range(1, max_C13_count+1):  #look for 1,2,3... C13
    
        c13_mass = m_mass + (c13_count * 1.003)   # expected c13 mass
        c13_name = f'{m_mass:.4f}(+{c13_count})'  # name is based on mono mass with (+1) etc apended
        
        while next_peak_index < len(peaks) - 1:   # -1 since we're going to increment it
            
            next_peak_index += 1  # point at next value in peak list
                
            next_peak_mass, next_peak_inten, next_peak_rt = peaks[next_peak_index]
            
            # mass is out of range  
            if next_peak_mass > (c13_mass + c13_half_window):
                keep_going = False       # when one isotope is not matched we abort and stop looking for more
                break
            
            # if the mass is in range we also check that the RT is within a window...
            # Note: it's OK to always apply this test since if there is no RT the values will be zero
            # and therefore the delta will be less than the threshold
            rt_ok = abs(m_rt-next_peak_rt)< c13_rt_window
            
            inten_ok = True if require_lower_c13_inten else next_peak_inten < m_inten
            
            if next_peak_mass > (c13_mass - c13_half_window) and rt_ok and inten_ok:
                c13 = Target(c13_mass, m.TargetIon.Root, c13_name)
                c13_matches.append(Match(next_peak_index, c13, peak_index, next_peak_mass - c13_mass))
                break     # and look for the next higher isotope
        
        if not keep_going:
            break     # leave 13c for loop   

matches += c13_matches

matches = sorted(matches, key = lambda x: x.Pk_index)  # sort by peak index...

matched_indices, percent_tic_matched = get_match_stats(matches, peaks, tic)

after_13c_match = f'{len(matched_indices)} peaks matched ({percent_tic_matched:.1f}% tic), {len(matches)} total matches from {len(peaks)} peaks'
print(after_13c_match)

122 peaks matched (26.9% tic), 126 total matches from 540 peaks


## Step 3 - Print, review, save

Cells illustrate various ways to report the matches:

- print all or some of them inside the notebook; there is an option to show all matches, including redundant ones, or to simplify the output to only show the shortest
- count the number of peaks that have redundant matches and optionally print them
- print the unmatched peaks above a thershold (as percentage of the base peak intensity)

Reviewing redundant peaks is useful as it can indicate that parameters need changing (if there are too many), for example: reduce the matching tolerance, reduce the number of target ions, etc.

The unmatched peak list is a good way to find peaks that still need to be explained and is the first step in further interpretation.

The detailed list os matched can be written to a file for use with the Interpret module or elsewhere and a final cell summarizes the parameters and results.

In [21]:
# Print the matches for review. To print a few matches use, for example, matches[:40]
# If there are redundant matches (i.e. more than one match for a peak) they are sorted by length, the idea being that that the simplest is more likely.
# If 'simplify' is True, only the first label (shortest) is printed with a notation indicating that there are others

print(after_13c_match)
print_match_list(matches[:40], peaks, match_as_str, simplify=True)  # remove '40' to see all

122 peaks matched (26.9% tic), 126 total matches from 540 peaks
   38:  205.1219 ( -0.4)   8.36        403.2   205.1223     38  Ibu_OH        Ibu_OH-H2O.H+
   39:  205.1220 ( -0.3)   6.80        468.7   205.1223     39  Ibu_OH        Ibu_OH-H2O.H+
   40:  205.1220 ( -0.3)   7.60       1715.0   205.1223     40  Ibu_OH        Ibu_OH-H2O.H+
   41:  206.1253 (  0.4)   8.36         55.9   206.1249     38  Ibu_OH        205.1219(+1)
   42:  206.1256 (  0.6)   7.60        267.9   206.1250     40  Ibu_OH        205.1220(+1)
   46:  207.1376 ( -0.4)   8.71        717.5   207.1380     46  Ibu           Ibu.H+
   50:  208.1406 (  0.0)   8.71         77.5   208.1406     46  Ibu           207.1376(+1)
   54:  219.1010 ( -0.6)   7.65        618.4   219.1016     54  Ibu_COOH      Ibu_COOH-H2O.H+
   55:  219.1011 ( -0.5)   6.27        121.9   219.1016     55  Ibu_COOH      Ibu_COOH-H2O.H+
   56:  221.1172 (  0.0)   6.09         40.4   221.1172     56  Ibu_(OH)2     Ibu_(OH)2-H2O.H+
   57:  221.1173 ( 

In [28]:
# We can also organize the matches by root before printing..
# Theh matches re sorted in peak (mass) order and isotopes can be skipped by seeting suppres_isotopes = True
# print_root_groups returns a dictionary which summarizes the results for each root group
#
# i.e. number of members, intensity and percent tic...if suppress_print is true we just get the diictionary
# if suppress_print is true the peaks are not printed and only the summary dictinary is reurned

grp_stats = print_root_groups(matches, peaks, match_as_str, suppress_isotopes = False, suppress_print=True)

for g in grp_stats:
    count, int_sum = grp_stats[g]
    percent_tic = int_sum * 100 / tic
    print(f'{g:12}\tcount:{count:4}\tinten sum {int_sum:12.1f}, {percent_tic:5.1f}% tic')

Ibu         	count:  14	inten sum       2415.8,   0.6% tic
Ibu_(OH)2   	count:   8	inten sum        939.8,   0.3% tic
Ibu_(OH)2_gluc	count:  10	inten sum       1709.4,   0.5% tic
Ibu_COOH    	count:  15	inten sum      11477.1,   3.1% tic
Ibu_COOH_gluc	count:  10	inten sum       4791.7,   1.3% tic
Ibu_OH      	count:  19	inten sum      12818.4,   3.4% tic
Ibu_OH_gluc 	count:  28	inten sum      36494.5,   9.8% tic
Ibu_gluc    	count:  22	inten sum      30179.9,   8.1% tic


In [23]:
# It can be useful to review the redundant matches, i.e. peaks that have multiple matches, since these can indicate adduct/loss/modifcation combinations
# that result in the same mass suggesting opprotunities to simplify the lists

redundant_peak_count, redundant_matches = get_redundant_matches(matches)

print(redundant_peak_count,' redundant peaks have', len(redundant_matches), 'matches')

print_redundant_matches = False

# To make the list easier to read, we add a blank line between the redundant groups
if print_redundant_matches:
    
    last_index = -1
    for m in redundant_matches:
        if m.Pk_index != last_index:   # when the Pk_index changes...
            print()
            last_index = m.Pk_index
            
        print(match_as_str(m, peaks))

4  redundant peaks have 8 matches


In [24]:
# It is useful to print the largest unmatched peaks since these suggest opportunities to modify the Calculator parameters
# to get greater coverage. Some peaks may be easy to explain (unusual isotopes, obvious losses, etc.)
# Unexplained, large peaks can be included in the Calculator, for example,  ('x544', 544.2148) was included in the Ibuprofen compound
# list to explain ions at 562 and 567 corresponding to X+NH4+ and X+Na+

threshold_percent = 1   #default threshold

bpi_percent_thresh = threshold_percent * base_peak_inten / 100   # convert to counts

unmatched_indices = get_unmatched_indices(matched_indices, peaks, bpi_percent_thresh)  # get the unmatched peask > 1% base peak

unmatched_mass, largest_unmatched_inten = 0,0

for pi in unmatched_indices:
    m,inten,rt = peaks[pi]
    percent_base_peak = inten * 100/ base_peak_inten
    print(f'{m:10.4f} {rt:8.2f} {inten:10.0f} {percent_base_peak:8.2f}% base peak')
    
    if inten > largest_unmatched_inten:
        unmatched_mass, largest_unmatched_inten = m,inten

large_inten_rel = largest_unmatched_inten * 100/ base_peak_inten

largest_unmatched_string = f'Largest unmatched {unmatched_mass}, {largest_unmatched_inten} {large_inten_rel:.1f}% base'

print(len(unmatched_indices), 'unmatched', largest_unmatched_string)

  177.1270     6.29        643     1.29% base peak
  222.1490     7.60        812     1.63% base peak
  224.1642     8.73       4248     8.53% base peak
  225.1675     8.71        756     1.52% base peak
  240.1594     7.62       8318    16.71% base peak
  241.1628     7.60       1775     3.57% base peak
  248.1494     2.01       1128     2.27% base peak
  254.1389     7.63      13349    26.81% base peak
  255.1422     7.63       2751     5.53% base peak
  256.1545     6.09        989     1.99% base peak
  258.1526     5.59        912     1.83% base peak
  350.2329     8.32       2709     5.44% base peak
  351.2363     8.32        748     1.50% base peak
  363.1441     6.98        735     1.48% base peak
  363.1441     6.16       3500     7.03% base peak
  364.1480     6.16        832     1.67% base peak
  398.1805     8.25       3355     6.74% base peak
  398.9490     8.71        599     1.20% base peak
  399.1840     8.25        998     2.00% base peak
  400.1965     8.73      45550 

In [25]:
# We make a list of strings containing the match conditions so they can be saved to the ouput files
# and printed (below)
match_conditions = [f'Match time: {curr_time_str}']
match_conditions.append(f'Peaks file: {peak_file_path}, {len(peaks)} peaks')
match_conditions.append(f'Matching amu half window: {amu_window}')
match_conditions.append(f'Matching ppm half window: {ppm_window} ppm')

match_conditions.append(f'Looking for <= {max_C13_count} 13C isotopes with half window {c13_half_window}')

match_conditions.append(initial_matches)
match_conditions.append(f'After 13C match {after_13c_match}')

match_conditions.append(f'{len(unmatched_indices)} unmatched peaks gt {threshold_percent}%, {largest_unmatched_string}')



In [26]:
# Save the match list if needed
# Note: The peak_file_path will already reflect the setting of 'local_files' so we don't need to test it again
# The code allows the largest unmatched peaks to be included if desired by creating empty matches for them
if save_matches:

    out_path, _ = os.path.splitext(peak_file_path)    # path without extension

    if include_date_in_file_name:
        out_path = f'{out_path} {compounds_as_string} matches {curr_time_str}.txt'
    else:
         out_path = f'{out_path} {compounds_as_string} matches.txt'       
    
    to_save = matches
    
    # If we want the unmatched ions, we generate an empty Match tuple for each and append them to the actual matches
    # Empty tuple has Pk_index, an empty target ion, the peak index is the mono peak, and an error of 0
    if include_large_unmatched:
        to_save = matches + [(i, Target(0,'', 'None'), i, 0) for i in unmatched_indices]
    
    to_save = sorted(to_save, key = lambda x: x[0])

    lines_written_count = save_matches_to_file(out_path, to_save, peaks, conditions, match_conditions, with_details=True)
    
    print(out_path)
    print(lines_written_count, ' lines written')

/Users/ronbonner/Data/SharedData/200325 SJ C18 Pos A, C001-U Ibu matches.txt
126  lines written


In [27]:
# Finally we summarize the results

print (current_time)

print(len(ions), 'targets')

if conditions:
    print_conditions(ion_file, conditions)

print()
for mc in match_conditions:
    print (mc)

2020-12-08 16:13:00
360 targets
Ibu ions pos 1.txt
Time:201208_092529
Compounds:Ibu
Multimer_limit:3
Polarity:positive
Phase_1:('OH', 2),('COOH', 1)
Phase_2:('gluc', 1)
Adducts:('Na-H', 2),('K-H', 2)
Max adduct count:6
Losses:('H2O', 1)

Match time: 201208_161300
Peaks file: /Users/ronbonner/Data/SharedData/200325 SJ C18 Pos A, C001-U.txt, 540 peaks
Matching amu half window: 0.005
Matching ppm half window: 20 ppm
Looking for <= 4 13C isotopes with half window 0.003
80 peaks matched (21.5% tic), 82 total matches from 540 peaks
After 13C match 122 peaks matched (26.9% tic), 126 total matches from 540 peaks
43 unmatched peaks gt 1%, Largest unmatched 416.1917, 49781.4 100.0% base
